In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt

# Fossil Fuel Subsidies

Note that we need to keep values titled "NA" because that's Namibia's ISO2 code. Also, Kosovo doesn't have an ISO3 so we can't use that.

In [ ]:
subsidy = pd.read_csv('input/policy/Fossil_Fuel_Subsidies.csv', index_col=0, keep_default_na=False, na_values="")
subsidy.head(3)

In [ ]:
country_nodes = subsidy.ISO2.unique()
policy_nodes = subsidy.CTS_Code.unique()
print("Number of Countries:", len(country_nodes))
print("Number of Subsidies:", len(policy_nodes))

Another question would be how do we create groups of subsidies. We'll call the fossil fuel subsidy bi-partite graph S.

In [ ]:
S = nx.Graph()
S.add_nodes_from(country_nodes, bipartite=0)
S.add_nodes_from(policy_nodes, bipartite=1)

Note that the following countries do not have the Petroleum field. We'll simply not draw edges.

In [ ]:
full = set(subsidy[subsidy.ISO2 == "AF"].CTS_Name)
subsidy_sub = subsidy[subsidy.Unit == "Percent of GDP"]
for country in country_nodes:
    subset = subsidy_sub[subsidy_sub.ISO2 == country]
    if len(subset) < 21:
        print(country, len(subset))
        partial = set(subsidy[subsidy.ISO2 == country].CTS_Name)
        print(full - partial)

Loop through every country/policy combination and add an edge if the value for specified year was greater than 0.

In [ ]:
print("Number of Edges:", S.number_of_edges())
print("Number of Possible Edges:", len(country_nodes) * len(policy_nodes))

### Change Point Detection

NEED TO FINISH

### Clustering

NEED TO FINSIH

# Green Bonds

### Intro

In [ ]:
green_bonds = pd.read_csv("input/policy/Green_Bonds.csv", index_col=0, keep_default_na=False, na_values="")

# Filtered Down to countries
green_bonds = green_bonds[np.logical_not(green_bonds["ISO2"].isna())]

green_bond_country_nodes = green_bonds.ISO2.unique()
# Need to figure out the differences between these bonds
bonds_nodes = green_bonds["CTS_Name"].unique()

Looks likes most years no one invested in bonds. Still very few countries invest in bonds but each year it seems the number increases.

In [ ]:
# Get the data
num_countries = green_bonds[green_bonds.columns[12:]].count().to_list()
years = [int(year[1:]) for year in green_bonds.columns[12:]]

# Plot
fig,ax = plt.subplots()
ax.scatter(years, num_countries)
ax.set_title("Number of Countries using Green Bonds by year")
ax.set_ylabel("Number of Countries")
ax.set_xlabel("Year")

plt.show()

Furthermore we can break the graph down into its two different categories

In [ ]:
# Get the data
issuances = green_bonds[green_bonds["CTS_Name"] == "Green Bonds Issuances"]


# Calculate Variables
issuance_count = issuances[issuances.columns[12:]].count()
nonissuance_count = num_countries - issuance_count
y = np.vstack([nonissuance_count, issuance_count])

# Plot
fig,ax = plt.subplots()
# Using years var from last cell
ax.stackplot(years, y, labels = ["Nonissuance Green Bonds", "Issuance Green Bonds",])
ax.set_title("Number of Countries using Green Bonds by year")
ax.set_ylabel("Number of Countries")
ax.set_xlabel("Year")
ax.legend(loc = "upper left")

plt.show()

In [ ]:
# Get the data
issuances = green_bonds[green_bonds["CTS_Name"] == "Green Bonds Issuances"]
nonissuances = green_bonds[green_bonds["CTS_Name"] == "Green Bonds"]

# Calculate Variables
issuance_money = issuances[issuances.columns[12:]].fillna(0).sum()
nonissuance_money = nonissuances[nonissuances.columns[12:]].fillna(0).sum()
y = np.vstack([nonissuance_money, issuance_money])

# Plot
fig,ax = plt.subplots()
# Using years var from last cell
ax.stackplot(years, y, labels = ["Nonissuance Green Bonds", "Issuance Green Bonds",])
ax.set_title("Number of Countries using Green Bonds by year")
ax.set_ylabel("Number of Countries")
ax.set_xlabel("Year")
ax.legend(loc = "upper left")

plt.show()

### Graph Building

Initializing the nodes in our bipartite graph. Country nodes are on the left and green bond policies are on the right.

In [ ]:
B = nx.Graph()
B.add_nodes_from(green_bond_country_nodes, bipartite=0)
B.add_nodes_from(bonds_nodes, bipartite=1)

We are creating the edge_lists for every single year. This can allow for quick creation of graphs for any specific year.

In [ ]:
green_bonds_temp = green_bonds.set_index(["ISO2", "CTS_Name"], inplace = False)
green_bond_years = []
green_bond_dict = {}

for year in green_bonds.columns[12:]: #create edge lists for each year
    edge_list = []
    for country in green_bond_country_nodes: #find every country-bond combo
        subset = green_bonds_temp.loc[country]
        for bond in subset.index: #bond
            weight = subset.loc[bond][year] #edge weights are bond values
            if weight > 0:
                edge = (country, bond, weight) #create edge
                edge_list.append(edge)
    # Take out years with no policies
    if not (len(edge_list) == 0):
        green_bond_years.append(int(year[1:]))
        green_bond_dict[int(year[1:])] = edge_list



Let's create a graph for 2015 and see what it looks like:

In [ ]:
B.remove_edges_from(B.edges)
B.add_weighted_edges_from(green_bond_dict[2015])
print("Number of Edges:", B.number_of_edges())
print("Number of Possible Edges:", len(green_bond_country_nodes) * len(bonds_nodes))

Let's also create a method to see the number of countries implementing any policies at all:

In [ ]:
def country_count(B, country_nodes):
    deg_policy, deg_country = bipartite.degrees(B, country_nodes)
    sum = 0
    for c, d in deg_country:
        if d > 0:
            sum += 1
    return sum
print("Participating Countries:", country_count(B, green_bond_country_nodes))

### Change Point Detection:

In [ ]:
windowSize = 6


def change_point_detection(windowSize, G, years, edge_dict):

    dict_signatures = {}
    for year in years:
        G.remove_edges_from(G.edges)
        G.add_weighted_edges_from(edge_dict[year])


        # Create the signless Laplacian but found the normalized laplacian to be better
        # A = nx.adjacency_matrix(B)
        # D = np.diag(nx.laplacian_matrix(B).diagonal())

        laplacian = nx.normalized_laplacian_matrix(G).toarray()
        singular_values = np.linalg.svd(laplacian)[1]
        l2_norm = np.linalg.norm(singular_values, ord = 2)
        norm_singluar_values = singular_values / l2_norm

        dict_signatures[year] = norm_singluar_values

    window_dicts = {}
    for i in range(len(years) - windowSize):
        year_subset = years[i:i+windowSize]

        # Choose one random signature to start our weights_avg
        weights_avg = np.zeros(dict_signatures[2018].shape)

        for year in year_subset:
            weights_avg += dict_signatures[year] * (1/windowSize)

        window_dicts[years[windowSize + i]] = weights_avg / np.linalg.norm(weights_avg, ord = 2)


    Z_list = []
    year_list = []

    for year in window_dicts.keys():
        baseline = window_dicts[year]
        single_year = dict_signatures[year]
        Z = 1-np.transpose(single_year) @ baseline

        Z_list.append(Z)
        year_list.append(year)

    return [Z_list, year_list]
    
Z_list, year_list = change_point_detection(windowSize, B, green_bond_years, green_bond_dict)

    


Successfully was able to show that the biggest change was in 2010s. Need to update method to take out laplacians that are 0, but I feel like this is important information.

In [ ]:
fig,ax = plt.subplots()
# Using years var from last cell
ax.plot(year_list, Z_list)
ax.set_title("Change Point Detection for Green Bonds \n Window of 6 years")
ax.set_ylabel("Change Value")
ax.set_xlabel("Year")

plt.show()


### Analysis

Temporal graph creation:

In [ ]:
results = []
for year in green_bond_years:
    B.remove_edges_from(B.edges)
    B.add_weighted_edges_from(dict[year])
    countries = country_count(B, green_bond_country_nodes) #participating countries
    results.append({"Year": year, "Countries": countries, "Edges": B.number_of_edges()})
pd.DataFrame(results)

Things to implement temporally: (loop through every year, show how these change)
- number of nodes (remove those without any edges)
- number of edges (weighted)
- number of trianges -> metamorphis coefficient
- change point detection
- any other correlation measures

Node based metrics:
- harmonic, closeness, degree, & eigenvector centralities
- you can show how it changes over time

Clustering:
- find out how to cluster bipartite graphs
- change point detection

### Clustering

This code maps each policy and country to its corresponding numerical id that will be used in the BINE code

In [ ]:
green_bond_node_mapping = {}

# Mapping for contries
i = 0
for country in green_bonds["ISO2"].unique():
    green_bond_node_mapping[country] = f"u{i}"
    i += 1

# Mapping for policies
j = 0
for policy in green_bonds["CTS_Name"].unique():
    green_bond_node_mapping[policy] = f"i{j}"
    j += 1

# Reverses the mapping to be id to name
inv_green_bond_mapping = {v: k for k, v in green_bond_node_mapping.items()}

There are 2 parts to the next code block:

1. First, we create edgelists for each graph in a format the BINE algorithm can read
1. Second we read each edgelist and create a command to run in the terminal that obtains the BINE embeddings

Note: The reason we have to write these commands to a new text file is because we need to run an old version of python for BINE algorithm to work.

In [ ]:
import os

# Create graphs BINE can read
for year in green_bond_years:
    B.remove_edges_from(B.edges)
    B.add_weighted_edges_from(green_bond_dict[year])

    # Relabels our graph from name to id to work on BINE code
    B_integer = nx.relabel_nodes(B, green_bond_node_mapping)

    fileName = f"BiNE-master/data/greenbonds/greenbonds_{year}_edgelist.dat"
    nx.write_weighted_edgelist(B_integer, fileName, delimiter = "\t")

# Create commands to run BINE file.
with open("Green_bond_commands.txt", "w") as f:
    for year in green_bond_years:
        command = f"python train.py --train-data ../data/greenbonds/greenbonds_{year}_edgelist.dat --lam 0.025 --d 10 --max-iter 100 --large 2 --vectors-u ../data/greenbonds/vectors_u_{year}.dat --vectors-v ../data/greenbonds/vectors_v_{year}.dat &&"
        f.write(command)


Now that we have feature representations of our countries we can complete a cluster analysis with them. 

Between the code block above and the code block below, we must run the commands file to obtain all the BINE embeddings. After we obtain the BINE embeddings, we then can read them into a pandas file for simple analysis.

In [ ]:
bine_embeddings_bonds_dict = {}

for year in green_bond_years:

    out_mat = []
    cols = []
    countries = []

    # This code is only opening the BINE embeddings for a specific year.
    with open(f"BiNE-master/data/greenbonds/vectors_u_{year}.dat", "r") as f:
        for line in f:
            data = line.split()

            # Parses what country our embedding is of.
            countries.append(data[0])

            # Obtains the embedding for country
            values = [float(elt) for elt in data[1:]]

            # Reshapes the embedding
            col = np.array(values).reshape((len(values),1))
            cols.append(col)

        out_mat = np.concatenate(cols, axis =1)

    # Changes ids of countries to their actual country names
    country_names = [inv_green_bond_mapping[country] for country in countries]

    pandas_bine_embeddings_bonds = pd.DataFrame(out_mat, columns = country_names)

    bine_embeddings_bonds_dict[year] = pandas_bine_embeddings_bonds

Lets view what each embedding looks like for each country

In [ ]:
pandas_bine_embeddings_bonds = bine_embeddings_bonds_dict[2021]
pandas_bine_embeddings_bonds

Before we officially cluster, I will use the Shilloute Score to see how many cluster we should use in our K-Means clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# First we obtain the clusterings on the BINE embeddings using K

X = pandas_bine_embeddings_bonds.transpose()

silscore_x = []
silscore_y = []

for k in range(2,40):
    clusterer = KMeans(n_clusters = k, random_state=10, n_init=10)
    clustering= clusterer.fit(X)
    score = silhouette_score(X, clustering.labels_)

    silscore_x.append(k)
    silscore_y.append(score)


fig, ax = plt.subplots()
scatter = ax.plot(silscore_x, silscore_y)
ax.set_title("Shilhouette Score Plot Green Bonds")
ax.set_ylabel("Shilhouette Score")
ax.set_xlabel("Number of Clusters")
plt.show()



From the above shilhouette scores it seems that the best number of clusters is 22, which means there will be about 3 countries per cluster. The clusters will then be pretty small. This also might be because the greenbonds is a pretty sparse dataset.

Now that we have examined a good clustering of the data, we can visualize our clutering. First we will try the optimal 22 clusters and then try a simplier one like 3 clusters.

As a baseline we can also look at the clustering if we just look at the column of each adjacency matrix

In [ ]:
columns = []
B.remove_edges_from(B.edges)
B.add_weighted_edges_from(green_bond_dict[2021])
countries_in_green_bonds = []

for node in B.nodes(data=True):

    # obtains nodes in partition 0, which is the country nodes
    if node[1]["bipartite"] == 0:

        col = np.zeros((2,1))
        # Obtains list of all edges that country is in
        for connect in B.edges(node[0], data = True):
            # Gets weight for Green Bonds Issuances
            if connect[1] == "Green Bonds Issuances":
                col[0,0] = connect[2]["weight"]
            # Gets weight for NonGreen Bonds Issuances
            else:
                col[1,0] = connect[2]["weight"]

        if not (col.sum() == 0):
            countries_in_green_bonds.append(node[0])
            columns.append(col)

sorted_indicies = np.argsort(np.array(countries_in_green_bonds))
graph_data = np.concatenate(columns, axis = 1)[:,sorted_indicies]

First we can make the clustering for the year 2021 using the normal clusters

In [ ]:
clustering_b = KMeans(n_clusters = 3, n_init= 10).fit(graph_data.transpose())
# clustering.labels_

fig,ax = plt.subplots()
# Using years var from last cell
scatter = ax.scatter(graph_data[0, :], graph_data[1,:], c = clustering_b.labels_)
ax.set_title("Countries Clustering in Greenbond Bond Investments in 2021 ")
ax.set_ylabel("Nonissuance Green Bonds")
ax.set_xlabel("Issuance Green Bonds")
legend1 = ax.legend(*scatter.legend_elements(),
                    loc="upper left", title="Classes")
ax.add_artist(legend1)

plt.show()

The following is helper code to get a mapping between the country name abbreviations and the full country name

In [ ]:
i = 0 
country_abv_to_full = {}
    
country_ab = green_bonds["ISO2"].unique()
country_list = green_bonds["Country"].unique()

for i in range(len(country_ab)):
    country_abv_to_full[country_ab[i]] = country_list[i]


This code takes the labels of the countries

In [ ]:
for abv in np.array(country_names)[clustering.labels_[np.argsort(np.array(country_names))] == 38]:
    print(country_abv_to_full[abv])

In [ ]:
bine_labels = np.array(clustering.labels_)[np.argsort(np.array(country_names))]
sorted_countries = np.array(country_names)[np.argsort(np.array(country_names))]
for abv in sorted_countries[bine_labels == 0]:
    print(country_abv_to_full[abv])

In [ ]:
relabeled_bine_labels = np.zeros(bine_labels.shape)

i = 0
for label in bine_labels:
    if label ==1 :
        relabeled_bine_labels[i] = 0
    elif label == 0:
        relabeled_bine_labels[i] = 1
    else:
        relabeled_bine_labels[i] =2 

    i+=1

for abv in sorted_countries[relabeled_bine_labels == 0]:
    print(country_abv_to_full[abv])


### Largest Change in Countries

Now that we have completed change point detection, we want to be able to analysis what caused the large change between years. For this we need to analysis how much a country changed between years. We can easily accomplish this by calculating the similarity between policies for countries in a range of years and find which country had the largest change between years. We don't use the bine embeddings of countries here because we don't have bine embeddings for every country. A country that is not implementing a policy will not have an embedding which causes challenges when calculating the change over time.

In [ ]:
yearly_adjacency_dict = {}
sorted_greenbond_countries = np.sort(green_bond_country_nodes)

for year in green_bond_years:
    columns = []
    B.remove_edges_from(B.edges)
    B.add_weighted_edges_from(green_bond_dict[year])
    countries_in_green_bonds = []

    for node in B.nodes(data=True):

        # obtains nodes in partition 0, which is the country nodes
        if node[1]["bipartite"] == 0:

            col = np.zeros((2,1))
            # Obtains list of all edges that country is in
            for connect in B.edges(node[0], data = True):
                # Gets weight for Green Bonds Issuances
                if connect[1] == "Green Bonds Issuances":
                    col[0,0] = connect[2]["weight"]
                # Gets weight for NonGreen Bonds Issuances
                else:
                    col[1,0] = connect[2]["weight"]

            countries_in_green_bonds.append(node[0])
            columns.append(col)

    sorted_indicies = np.argsort(np.array(countries_in_green_bonds))
    graph_data = np.concatenate(columns, axis = 1)[:,sorted_indicies]
    

    yearly_adjacency_dict[year] = graph_data



Now that we have a dataframe that stores all of the matrices, we can now compute the differences between years with a given window size.

In [ ]:
window_dicts = {}

def country_difference(windowSize, adjacency_dict, years, countries):
    
    window_dicts = {}
    for i in range(len(years) - windowSize):
        year_subset = years[i:i+windowSize]

        # Choose one random signature to start our weights_avg
        weights_avg = np.zeros(adjacency_dict[years[0]].shape)

        for year in year_subset:
            weights_avg += adjacency_dict[year] * (1/windowSize)

        window_dicts[years[windowSize + i]] = weights_avg / np.linalg.norm(weights_avg, ord = 2)


    country_change = {}

    for year in window_dicts.keys():
        baseline = window_dicts[year]
        single_year = adjacency_dict[year]
        
        change = np.power(baseline - single_year, 2).sum(axis = 0)
        sort_indices = np.argsort(change)[::-1]
        info = (countries[sort_indices], change[sort_indices])

        country_change[year] = info

    return country_change


country_change = country_difference(6,yearly_adjacency_dict, green_bond_years, sorted_greenbond_countries)


In [ ]:
year = 2014

countryOrder_abv, changeScore = country_change[year]

countryOrder = [country_abv_to_full[country] for country in countryOrder_abv[changeScore > 0]]
changeScore = changeScore[changeScore > 0]

fig, ax = plt.subplots()
ax.barh(countryOrder[:15], changeScore[:15])
ax.set_title(f"Most Change in Policies in {year}")
ax.set_ylabel("Countries")
ax.set_xlabel("Change (Euclidean Distance)")
plt.show()

In [ ]:
countryOrder_abv

In [ ]:
green_bonds[green_bonds["ISO2"] == "CN"][["ISO2","Indicator","F2012","F2013", "F2014", "F2015", "F2016", "F2017"]]


In [ ]:
green_bonds[green_bonds["ISO2"] == "CN"][["ISO2","Indicator","F2012","F2013", "F2014", "F2015", "F2016", "F2017"]]

In [ ]:
green_bonds[green_bonds["ISO2"] == "US"][["ISO2","Indicator","F2012","F2013", "F2014", "F2015"]]

# Environmental Taxes

First lets load in the data and take a quick peak.  The final table is a table about how many measures each country implements.

In [ ]:
taxes = pd.read_csv("input/policy/Environmental_Taxes.csv")

num_countries = len(taxes["Country"].unique())
print(f"There are {num_countries} unique countries that are implementing environmental taxes")
print("Each country typically reports 10 different measures, but there are a few that report less than 10 measures")

taxes.Country.value_counts().value_counts()

In [ ]:
tax_country_nodes = taxes.ISO2.unique()
tax_policy_nodes = taxes.CTS_Code.unique()
print("Number of Countries:", len(country_nodes))
print("Number of Subsidies:", len(policy_nodes))

T = nx.Graph()
T.add_nodes_from(country_nodes, bipartite=0)
T.add_nodes_from(policy_nodes, bipartite=1)

In [ ]:
tax_edge_dict = {}

taxes_sub = taxes[taxes.Unit == "Percent of GDP"]
for year in taxes_sub.columns[10:]:
    edge_list = []
    for country in tax_country_nodes:
        subset = taxes_sub[taxes_sub.ISO2 == country]
        for policy in tax_policy_nodes:
            weight = subset[subset.CTS_Code == policy][year].values
            if weight > 0:
                edge = (country, policy, weight[0])
                edge_list.append(edge)
    tax_edge_dict[int(year[1:])] = edge_list

In [ ]:
T.add_weighted_edges_from(edges)
print("Number of Edges:", T.number_of_edges())
print("Number of Possible Edges:", len(country_nodes) * len(policy_nodes))

### Analysis

In [ ]:
taxes_no_zeros = taxes_sub.fillna(0)
graph_years = []
graph_count = []
graph_country_count = []

for year in taxes_sub.columns[10:]:
    implement_count = (taxes_no_zeros[year] > 0).sum()

    b = taxes_no_zeros[year] > 0
    countries = taxes[taxes["Unit"] == "Percent of GDP"]["ISO2"]
    num_countries = len(countries[b].unique())

    graph_years.append(int(year[1:]))
    graph_count.append(implement_count)
    graph_country_count.append(num_countries)

# Plot
fig,ax = plt.subplots()
# Using years var from last cell
ax.plot(graph_years, graph_count)
ax.set_title("Number of Tax Policies being Implemented")
ax.set_ylabel("Number of Policies Implemented")
ax.set_xlabel("Year")

plt.show()

# Plot
fig,ax = plt.subplots()
# Using years var from last cell
ax.plot(graph_years, graph_country_count)
ax.set_title("Number of Countries Implementing Tax Policies")
ax.set_ylabel("Number of Countries implementing")
ax.set_xlabel("Year")

plt.show()


Another simple analysis I want to have is how many countries are implementing tax policies each year.

In [ ]:
fig,ax = plt.subplots()
# Using years var from last cell
ax.plot(year_list, Z_list)
ax.set_title("Change Point Detection for Green Bonds \n Window of 6 years")
ax.set_ylabel("Change Value")
ax.set_xlabel("Year")

plt.show()

### Change Point 

In [ ]:
tax_years = []
for key in tax_edge_dict.keys():
    tax_years.append(key)

windowSize = 2
Z_list, year_list = change_point_detection(windowSize, T, tax_years, tax_edge_dict)

fig,ax = plt.subplots()
# Using years var from last cell
ax.plot(year_list, Z_list)
ax.set_title("Change Point Detection Taxes")
ax.set_ylabel("Change Value")
ax.set_xlabel("Year")

plt.show()

### Clustering

In [ ]:
taxes_node_mapping = {}

i = 0
for country in taxes["ISO2"].unique():
    taxes_node_mapping[country] = f"u{i}"
    i += 1


j = 0
for policy in taxes["CTS_Code"].unique():
    taxes_node_mapping[policy] = f"i{j}"
    j += 1

inv_taxes_mapping = {v: k for k, v in taxes_node_mapping.items()}

In [ ]:
# Create files 
for year in tax_years:
    T.remove_edges_from(T.edges)
    T.add_weighted_edges_from(tax_edge_dict[year])

    T_integer = nx.relabel_nodes(T, taxes_node_mapping)

    fileName = f"BiNE-master/data/taxes/taxes_{year}_edgelist.dat"
    nx.write_weighted_edgelist(T_integer, fileName, delimiter = "\t")

with open("Taxes_commands.txt", "w") as f:
    for year in tax_years:
        command = f"python train.py --train-data ../data/taxes/taxes_{year}_edgelist.dat --lam 0.025 --max-iter 100 --large 2 --vectors-u ../data/taxes/vectors_u_{year}.dat --vectors-v ../data/taxes/vectors_v_{year}.dat &&"
        f.write(command)

# Environmental Protection Expenditures

In [ ]:
expenditures = pd.read_csv("input/policy/Environmental_Protection_Expenditures.csv", index_col=0, keep_default_na=False, na_values="")
expenditures.head(5)

In [ ]:
exp_country_nodes = expenditures.ISO2.unique()
exp_policy_nodes = expenditures.CTS_Code.unique()
print("Number of countries:", len(country_nodes))
print("Number of expenditures:", len(policy_nodes))

E = nx.Graph()
E.add_nodes_from(country_nodes, bipartite=0)
E.add_nodes_from(policy_nodes, bipartite=1)

In [ ]:
expenditure_edge_dict = {}

exp_sub = expenditures[expenditures.Unit == "Percent of GDP"]
for year in exp_sub.columns[9:]:
    edge_list = []
    for country in exp_country_nodes:
        subset = exp_sub[exp_sub.ISO2 == country]
        for policy in exp_policy_nodes:
            weight = subset[subset.CTS_Code == policy][year].values
            if weight.size > 0 and weight > 0:
                edge = (country, policy, weight[0])
                edge_list.append(edge)
    expenditure_edge_dict[int(year[1:])] = edge_list
    


In [ ]:
print("Number of possible edges:", len(country_nodes) * len(policy_nodes))
print("Number of edges:", len(edges))

### Change Point Detection

In [ ]:
expenditures_years = []
for key in expenditure_edge_dict.keys():
    expenditures_years.append(key)

windowSize = 3
Z_list, year_list = change_point_detection(windowSize, E, expenditures_years, expenditure_edge_dict)


fig,ax = plt.subplots()
# Using years var from last cell
ax.plot(year_list, Z_list)
ax.set_title("Change Point Detection Expenditures")
ax.set_ylabel("Change Value")
ax.set_xlabel("Year")

plt.show()

### Analysis

In [ ]:
for comp in nx.connected_components(B):
    print(comp)

### Clustering

In [ ]:
expenditures_node_mapping = {}

i = 0
for country in expenditures["ISO2"].unique():
    expenditures_node_mapping[country] = f"u{i}"
    i += 1


j = 0
for policy in expenditures["CTS_Code"].unique():
    expenditures_node_mapping[policy] = f"i{j}"
    j += 1

inv_taxes_mapping = {v: k for k, v in expenditures_node_mapping.items()}

In [ ]:
# Create files 
for year in expenditures_years:
    E.remove_edges_from(E.edges)
    E.add_weighted_edges_from(expenditure_edge_dict[year])

    E_integer = nx.relabel_nodes(E, expenditures_node_mapping)

    fileName = f"BiNE-master/data/expenditures/expenditures_{year}_edgelist.dat"
    nx.write_weighted_edgelist(E_integer, fileName, delimiter = "\t")

# Writes command line prompt to file
with open("Expenditures_commands.txt", "w") as f:
    for year in tax_years:
        command = f"python train.py --train-data ../data/expenditures/expenditures_{year}_edgelist.dat --lam 0.025 --max-iter 100 --large 2 --vectors-u ../data/expenditures/vectors_u_{year}.dat --vectors-v ../data/expenditures/vectors_v_{year}.dat &&"
        f.write(command)